In [ ]:
!pip install gdown

In [ ]:
!git clone https://github.com/pytorch/vision.git
%cd vision
!git checkout v0.3.0

!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../

%cd ..

In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip install timm

In [ ]:
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu102

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1maXjFAFcPhXG8B4F1emu_cLq6X06OW_p'

output = 'dataset.zip'
gdown.download(url, output)

In [ ]:
!unzip dataset.zip >/dev/null

**References** <br>
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html <br>
https://haochen23.github.io/2020/06/fine-tune-mask-rcnn-pytorch.html <br>
https://discuss.pytorch.org/t/multi-class-implementation-mask-rcnn/70476

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import transforms as T


class DocumentDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        if self.transforms is not None:
            img_np = np.array(img)
            mask_np = np.array(mask)
            transformed = self.transforms(image=img_np, mask=mask_np)
            img = Image.fromarray(transformed["image"])
            mask = Image.fromarray(transformed["mask"])
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        # labels = torch.ones((num_objs,), dtype=torch.int64)
        # there are now 11 classes (background + 10 documents)
        labels = torch.as_tensor(obj_ids, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        trans = T.Compose([T.ToTensor()])
        img, target = trans(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.rpn import AnchorGenerator
# import timm
# import torch

def build_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    # Available in nightly version
    # model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Stop here if you are fine-tunning Faster-RCNN

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

# def build_model(num_classes):
#     # load a pre-trained model for classification and return
#     # only the features
#     # m = timm.create_model('resnet50', pretrained=True)
#     # backbone = torch.nn.Sequential(*(list(m.children())[:-2]))
#     # backbone = torchvision.models.efficientnet_b4(pretrained=True).features
#     # backbone = timm.create_model('resnet50', features_only=True, pretrained=True)
#     # backbone.out_channels = 2048
    
#     backbone = timm.create_model('resnet50', pretrained=True)
#     backbone.reset_classifier(0, '')
#     backbone.out_channels = backbone.num_features

#     # let's make the RPN generate 5 x 3 anchors per spatial
#     # location, with 5 different sizes and 3 different aspect
#     # ratios. We have a Tuple[Tuple[int]] because each feature
#     # map could potentially have different sizes and
#     # aspect ratios
#     anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                        aspect_ratios=((0.5, 1.0, 2.0),))

#     # let's define what are the feature maps that we will
#     # use to perform the region of interest cropping, as well as
#     # the size of the crop after rescaling.
#     # if your backbone returns a Tensor, featmap_names is expected to
#     # be [0]. More generally, the backbone should return an
#     # OrderedDict[Tensor], and in featmap_names you can choose which
#     # feature maps to use.
#     roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
#                                                     output_size=7,
#                                                     sampling_ratio=2)
    
#     mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
#                                                          output_size=14,
#                                                          sampling_ratio=2)

#     # put the pieces together inside a MaskRCNN model
#     model = MaskRCNN(backbone,
#                      num_classes=num_classes,
#                      rpn_anchor_generator=anchor_generator,
#                      box_roi_pool=roi_pooler,
#                      mask_roi_pool=mask_roi_pooler)
    
#     return model

In [ ]:
from engine import train_one_epoch, evaluate
import os
import utils
# import torchvision.transforms as T
import albumentations as A


def get_transform(train):
    transforms = []
    # data augmentation
    if train:
        transforms.append(A.Rotate(interpolation=0, border_mode=0, value=0, mask_value=0))
    return A.Compose(transforms)

# use our dataset and defined transformations
dataset = DocumentDataset(os.path.join('dataset', 'train'), get_transform(train=True))
dataset_test = DocumentDataset(os.path.join('dataset', 'test'), get_transform(train=False))

# split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=os.cpu_count(),
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=os.cpu_count(),
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 11 classes - background and 10 documents
num_classes = 11

# get the model using our helper function
model = build_model(num_classes)
# move model to the right device
model.to(device)
print(model)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
# number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    result = evaluate(model, data_loader_test, device=device)

In [ ]:
torch.save(model, 'mask-rcnn-document.pt')

In [ ]:
!rm mask-rcnn-document.pt

<a href="./mask-rcnn-document.pt"> Download Model</a>